# Jupyter meets MathBox2

Apart from being a standalone mathematical visualization libary
MathBox2 can be used as a powerful data presentation tool in
variuos data analysis pipelines. 

This notebook contains a few examples of embedded MathBox2 plots.

In [33]:
import json
import numpy as np
from IPython.display import HTML, Javascript, display

In [34]:
def json_numpy_serialzer(o):
    if isinstance(o, np.ndarray):
        return o.tolist()
    raise TypeError("{} of type {} is not JSON serializable".format(repr(o), type(o)))

def jsglobal(**params):
    '''Populate JS global namespace with provided Python obejcts.'''
    code = [];
    for name, value in params.items():
        jsdata = json.dumps(value, default=json_numpy_serialzer)
        code.append("window.{} = {};".format(name, jsdata))
    display(Javascript("\n".join(code)))

In [35]:
%%javascript

require.config({
    paths: {
        mathBox: '//cdn.rawgit.com/unconed/mathbox/eaeb8e15/build/mathbox-bundle'
        //mathBox: '//localhost:8000/build/mathbox-bundle'
    }
});

window.with_mathbox = function(element, func) {
    require(['mathBox'], function(){
        var mathbox = mathBox({
          plugins: ['core', 'controls', 'cursor', 'mathbox'],
          controls: { klass: THREE.OrbitControls },
          mathbox: {inspect: false},
          element: element[0],
        });
        var three = mathbox.three;
        three.renderer.setClearColor(new THREE.Color(0xFFFFFF), 1.0);
        three.camera.position.set(-1, 1, 2);
        three.controls.noKeys = true;
        
        element[0].style.height = "400px";
        element[0].style.width = "100%";
        window.dispatchEvent(new Event('resize'));
        
        func(mathbox);
    })
}

<IPython.core.display.Javascript object>

In [36]:
%%javascript
with_mathbox(element, function(mathbox) {
   mathbox.cartesian()
     .area({width:50, height:50,
            expr: function(emit, x, y, i, j, t){
              var r = Math.sqrt(x*x+y*y);
              var z = Math.sin(r*10-t*0.5)*0.2;
              emit(x, z, y);
           }})
     .surface({color:'#AAA', shaded:true})
     .surface({color:'#55A', lineX:true, lineY:true, fill:false, zBias:1});
})

<IPython.core.display.Javascript object>

In [37]:
%%javascript
with_mathbox(element, function(mathbox) {
  element[0].style.height = '600px';
  mathbox.cartesian().grid({width: 2, opacity: 0.5, axes: [1, 3], origin: [0, -1, 0]});

  var view = mathbox.stereographic4({position:[0, 0, 0], scale:[0.5, 0.5, 0.5]});

  var edges = [];
  var points = []
  for (var e=-1; e<2; e+=2)
  for (var i=-1; i<2; i+=2)
  for (var j=-1; j<2; j+=2)
  for (var k=-1; k<2; k+=2) {
    edges.push([i, j, k, e])
    edges.push([i, j, e, k])
    edges.push([i, e, j, k])
    edges.push([e, i, j, k])
    points.push([i, j, k, e])
  }

  view.matrix({width:edges.length/2, height:2, data:edges, live: false})
  .transpose({order:"yx", id:"edges"})
  .array({data:points, id:"points"})
  .clock({speed:0.25})
  .transform4({}, {matrix:function(t) {
     var c = Math.cos(t), s = Math.sin(t);
     return [c, 0, 0,-s,
             0, 1, 0, 0,
             0, 0, 1, 0,
             s, 0, 0, c];
     }})  
    .point({size:8, points:"#points"})
    .format({live:false, expr:(x, y, z, w)=>{
      return x+", "+y+", "+z+", "+w;
    }}).label({size:16, depth:0.5, outline:0.5})
    .lerp({width:32, source:"#edges"})
    .line({color:0x3090FF, depth:1.0, width:4});
})

<IPython.core.display.Javascript object>